In [22]:
import os
import cv2
import mediapipe as mp
import pandas as pd

In [23]:
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True, max_num_faces=1, refine_landmarks=True, min_detection_confidence=0.5)

In [24]:
base_dir = "C:/Users/91727/OneDrive/Desktop/dts/train"

In [25]:
landmarks_data = []

In [26]:
def extract_landmarks1(image):
    # Convert the BGR image to RGB
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Process the image and extract landmarks
    results = face_mesh.process(image_rgb)
    
    # Check if landmarks are detected
    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            # Flatten x, y, z coordinates into a single list
            flattened_landmarks = []
            for landmark in face_landmarks.landmark:
                flattened_landmarks.extend([landmark.x, landmark.y, landmark.z])  # Append x, y, z coordinates

            return flattened_landmarks

    return None

In [27]:
def extract_landmarks2(image):
    # Convert the BGR image to RGB
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Process the image and extract landmarks
    results = face_mesh.process(image_rgb)
    
    # Check if landmarks are detected
    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            # Extract x, y, z coordinates into lists
            xs = [landmark.x for landmark in face_landmarks.landmark]
            ys = [landmark.y for landmark in face_landmarks.landmark]
            zs = [landmark.z for landmark in face_landmarks.landmark]
            
            # Normalize x, y, and z coordinates
            min_x, min_y, min_z = min(xs), min(ys), min(zs)
            normalized_landmarks = []

            for x, y, z in zip(xs, ys, zs):
                normalized_landmarks.append(x - min_x)  # Normalize x
                normalized_landmarks.append(y - min_y)  # Normalize y
                normalized_landmarks.append(z - min_z)  # Normalize z

            return normalized_landmarks

    return None

'def extract_landmarks(image):\n    # Convert the BGR image to RGB\n    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)\n    \n    # Process the image and extract landmarks\n    results = face_mesh.process(image_rgb)\n    \n    # Check if landmarks are detected\n    if results.multi_face_landmarks:\n        for face_landmarks in results.multi_face_landmarks:\n            # Extract x, y, z coordinates into lists\n            xs = [landmark.x for landmark in face_landmarks.landmark]\n            ys = [landmark.y for landmark in face_landmarks.landmark]\n            zs = [landmark.z for landmark in face_landmarks.landmark]\n            \n            # Normalize x, y, and z coordinates\n            min_x, min_y, min_z = min(xs), min(ys), min(zs)\n            normalized_landmarks = []\n\n            for x, y, z in zip(xs, ys, zs):\n                normalized_landmarks.append(x - min_x)  # Normalize x\n                normalized_landmarks.append(y - min_y)  # Normalize y\n                

In [28]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module='google.protobuf')

In [29]:
for category in ["angry", "disgusted", "happy", "neutral", "sad", "surprised"]:
    category_path = os.path.join(base_dir, category)
    
    for image_file in os.listdir(category_path):
        image_path = os.path.join(category_path, image_file)
        image = cv2.imread(image_path)
        
        # Extract landmarks from the image
        landmarks = extract_landmarks(image)
        
        if landmarks:
            # Add the category and landmarks to the data list
            landmarks_data.append({'category': category, 'landmarks': landmarks})

In [30]:
df_landmarks = pd.DataFrame(landmarks_data)

In [31]:
df_landmarks.to_csv('new_landmarks_nn.csv', index=False)